In [ ]:
import pandas as pd
import numpy as np

# load dataset from github link, create pandas dataframe
# url = 'https://raw.githubusercontent.com/Keerat-Grewal/KDD_Project/main/Correct_Dataset.csv'
# url = 'https://raw.githubusercontent.com/Keerat-Grewal/KDD_Project/main/cardio_train.csv'
url = 'https://raw.githubusercontent.com/Keerat-Grewal/KDD_Project/main/heart.csv'
hd_df = pd.read_csv(url)
# hd_df.drop(columns=hd_df.columns[0], axis=1, inplace=True)
# hd_df.loc[:,'age']=hd_df['age'].fillna(hd_df['age'].median())
# hd_df.loc[:,'age']=(hd_df['age']/10).astype(str).str[0].astype(int)*10

hd_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
# sklearn bayessian classifier
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split


# def train_test_split(X,y,test_frac=0.5):
#   inxs = list(range(len(y)))
#   n = len(inxs)
#   nTest=int(test_frac*n)
#   np.random.shuffle(inxs)
#   X = X.iloc[inxs,:]
#   y = y.iloc[inxs]
#   Xtrain,ytrain,Xtest,ytest = X[:n-nTest], y[:n-nTest], X[n-nTest:], y[n-nTest:]
#   return np.asarray(Xtrain).astype(np.float32),np.asarray(ytrain).astype(np.float32),np.asarray(Xtest).astype(np.float32),np.asarray(ytest).astype(np.float32)
# def clean_data(df):
#   cols = list(df.columns)
#   for col in cols:
#     df = df[df[col] != "?"]
#   return df

dataY = hd_df["target"]
dataX = hd_df.drop(["target"], axis=1)

best_accuracy = None
best_random_state = None
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.20, random_state=31)
bayessian_model = GaussianNB()
bayessian_model.fit(X_train, y_train)
accuracy_sk = bayessian_model.score(X_test, y_test)
print(accuracy_sk)

0.9016393442622951


$\Pr(target = 1|trestbps, chol, etc..)=\frac{\Pr(trestbps|target = 1)*\Pr(chol|target = 1),etc.*\Pr(target = 1)}{\Pr(trestbps|target = 1)*\Pr(chol|target = 1),etc.*\Pr(target = 1)+\Pr(trestbps|\neg target = 1)*\Pr(chol|\neg target = 1),etc.*Pr(\neg target = 1)}$


In [ ]:
# Our own bayessian classifier

# calculate priors for target = 1 or target = 0
priors = {}
target_values = hd_df['target'].value_counts()

for i in range(2):
  priors[f'target={i}'] = target_values[i] / len(hd_df['target'])

priors

{'target=0': 0.45544554455445546, 'target=1': 0.5445544554455446}

In [ ]:
# get standard deviation and means for each one

target_to_stat = {}

for i in hd_df['target'].unique():
    filtered_df = hd_df[lambda x : x.target == i]
    target_to_stat[f'{i}_std'] = filtered_df.std()
    target_to_stat[f'{i}_mean'] = filtered_df.mean()

target_to_stat

{'0_mean': age          56.601449
 sex           0.826087
 cp            0.478261
 trestbps    134.398551
 chol        251.086957
 fbs           0.159420
 restecg       0.449275
 thalach     139.101449
 exang         0.550725
 oldpeak       1.585507
 slope         1.166667
 ca            1.166667
 thal          2.543478
 target        0.000000
 dtype: float64, '0_std': age          7.962082
 sex          0.380416
 cp           0.905920
 trestbps    18.729944
 chol        49.454614
 fbs          0.367401
 restecg      0.541321
 thalach     22.598782
 exang        0.499232
 oldpeak      1.300340
 slope        0.561324
 ca           1.043460
 thal         0.684762
 target       0.000000
 dtype: float64, '1_mean': age          52.496970
 sex           0.563636
 cp            1.375758
 trestbps    129.303030
 chol        242.230303
 fbs           0.139394
 restecg       0.593939
 thalach     158.466667
 exang         0.139394
 oldpeak       0.583030
 slope         1.593939
 ca            0.

In [ ]:
import scipy.stats

# def specific_class_conditional(feature_name,feature_val,given_name,given_val):
#     attrs = ['trestbps', 'chol', 'thalach', 'oldpeak']
#     if not isinstance(feature_name, pd.Series):
#       mean = target_to_stat[str(given_val)+'_mean'][feature_name]
#       std = target_to_stat[str(given_val)+'_std'][feature_name]
#       prob = scipy.stats.norm.pdf(feature_val,mean,std)
#     else:
#       # x,xv,y,yv
#       filtered = given_name.loc[lambda item: item == given_val]
#       count = 0
    
#       for i in filtered.index:
#           if feature_name.loc[i] == feature_val:
#               count += 1
      
#       prob = count / len(filtered)
    
#     return prob


def specific_class_conditional(feature_name,feature_val,given_name,given_val):
    mean = target_to_stat[str(given_val)+'_mean'][feature_name]
    std = target_to_stat[str(given_val)+'_std'][feature_name]
    prob = scipy.stats.norm.pdf(feature_val,mean,std)
    
    return prob


    
# specific_class_conditional('age', 30, 'target', '1' )

In [ ]:
def class_conditional(X,y):
    probs = {}
    for i in X.columns:
        unique = X[i].unique()
        for j in unique:
            for target in y.unique():
                get_prob = specific_class_conditional(X[i], j, y, target)
                probs[f'{i}={float(j)}|target={target}'] = get_prob
        
    return probs

probs = class_conditional(X_train, y_train)
probs

{'age=29.0|target=0': 0.0,
 'age=29.0|target=1': 0.007194244604316547,
 'age=34.0|target=0': 0.0,
 'age=34.0|target=1': 0.014388489208633094,
 'age=35.0|target=0': 0.019417475728155338,
 'age=35.0|target=1': 0.014388489208633094,
 'age=37.0|target=0': 0.0,
 'age=37.0|target=1': 0.014388489208633094,
 'age=38.0|target=0': 0.0,
 'age=38.0|target=1': 0.014388489208633094,
 'age=39.0|target=0': 0.009708737864077669,
 'age=39.0|target=1': 0.02158273381294964,
 'age=40.0|target=0': 0.009708737864077669,
 'age=40.0|target=1': 0.007194244604316547,
 'age=41.0|target=0': 0.009708737864077669,
 'age=41.0|target=1': 0.05755395683453238,
 'age=42.0|target=0': 0.009708737864077669,
 'age=42.0|target=1': 0.04316546762589928,
 'age=43.0|target=0': 0.02912621359223301,
 'age=43.0|target=1': 0.02877697841726619,
 'age=44.0|target=0': 0.019417475728155338,
 'age=44.0|target=1': 0.050359712230215826,
 'age=45.0|target=0': 0.019417475728155338,
 'age=45.0|target=1': 0.04316546762589928,
 'age=46.0|target=

In [ ]:
def posteriors(probs,priors,x):
    post_probs = {}

    # extract info
    attrs = ['trestbps', 'chol', 'thalach', 'oldpeak']

    # calculate probs for each fruit_name
    for i in hd_df['target'].unique():

        num = 1
        try:
            for attr in list(hd_df.columns)[:-1]:
                val = x[attr]
                # if attr in attrs:
                #   num *= specific_class_conditional(attr, val, 'target', i) 
                # else:
                #   num *= probs[f'{attr}={float(val)}|target={i}']
                num *= specific_class_conditional(attr, val, 'target', i) 
            num *= priors[f'target={i}']
            denom = num
            other_target = [j for j in hd_df['target'].unique() if i != j]
            
            assert len(other_target) == len(hd_df['target'].unique()) - 1
            
            for target in other_target:
                next_val = 1
                for attr2 in list(hd_df.columns)[:-1]:
                    res = x[attr2]
                    # if attr2 in attrs:
                    #   next_val *= specific_class_conditional(attr2, res, 'target', target)
                    # else:
                      # next_val *= probs[f'{attr2}={float(res)}|target={target}']
                    next_val *= specific_class_conditional(attr2, res, 'target', target)
                next_val *= priors[f'target={target}']
                denom += next_val
        except:
            print('Key error')
            num = 1
            denom = 2

        string = f'target={i}|'
        for j in list(hd_df.columns)[:-1]:
          real_val = x[j]
          string += f'{j}={real_val},'
        post_probs[string[:-1]] = num/denom
    return post_probs

ans = posteriors(probs, priors, X_test.iloc[0])

In [ ]:
def predict(X_train, X_test, y_train, y_test):
    accuracy = None
    
    count = 0
    probs = class_conditional(X_train, y_train)
        
    for i in X_test.index:
        # ans = posteriors({}, priors, X_test.loc[i])
        ans = posteriors({}, priors, X_test.loc[i])
        for k,v in ans.items():
            if v > 0.5:
                curr_target = k[k.find('=')+1:k.find('|')]
                if y_test.loc[i] == int(curr_target):
                    count += 1

    accuracy = count / len(y_test)
    return accuracy


final_accuracy = predict(X_train, X_test, y_train, y_test)
print('final accuracy:', final_accuracy)

final accuracy: 0.9016393442622951


In [ ]:
def feature_importance(Xtrain,ytrain,Xtest,ytest, npermutations = 5):
    # initialize what we are going to return
    importances = {}
    for col in Xtrain.columns:
        importances[col] = 0
    # find the original accuracy
    orig_accuracy = predict(Xtrain,Xtest,ytrain,ytest)
    # now carray out the feature importance work
    for col in Xtrain.columns:
        for perm in range(npermutations):
            Xtest2 = Xtest.copy()
            Xtest2[col] = Xtest[col].sample(frac=1, replace=False).values
            error = orig_accuracy - predict(Xtrain,Xtest2,ytrain,ytest)
            importances[col] += error
            
        importances[col] = importances[col]/npermutations

    return importances

feature_importance(X_train,y_train,X_test,y_test)

{'age': -0.02622950819672132,
 'ca': 0.009836065573770503,
 'chol': -0.0360655737704918,
 'cp': -0.006557377049180335,
 'exang': 0.02295081967213115,
 'fbs': 0.0032786885245901674,
 'oldpeak': 0.019672131147540982,
 'restecg': 0.0,
 'sex': -0.006557377049180335,
 'slope': 0.009836065573770503,
 'thal': 0.022950819672131174,
 'thalach': 0.006557377049180335,
 'trestbps': -0.032786885245901634}

thal, exang, oldpeak, ca ==> most important features